In [248]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [249]:
import pandas as pd
import numpy as np
import pickle

In [250]:
df_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Datarisk/data/base_pagamentos_teste.csv')

In [251]:
df_cadastro = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Datarisk/data/base_cadastral.csv')

In [252]:
df_info = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Datarisk/data/base_info.csv')

In [253]:
df_raw = df_test.merge( df_cadastro, how='left', on='ID_CLIENTE')

In [254]:
test = df_raw.merge( df_info, how='left', right_on=['ID_CLIENTE','SAFRA_REF'], left_on=['ID_CLIENTE','SAFRA_REF'])

In [255]:
test

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,Unnamed: 0_x,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,Unnamed: 0_y,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
0,5058298901476893676,2021-07,2021-07-14,2021-08-04,11204.75,4.99,641,2014-03-30,54,NaN,Serviços,GMAIL,MEDIO,99,132986.0,467430.0,154.0
1,274692171162531764,2021-07,2021-07-08,2021-08-23,60718.50,5.99,252,2005-08-03,19,NaN,Serviços,GMAIL,PEQUENO,13,420995.0,417192.0,104.0
2,274692171162531764,2021-07,2021-07-11,2021-08-25,60718.50,5.99,252,2005-08-03,19,NaN,Serviços,GMAIL,PEQUENO,13,420995.0,417192.0,104.0
3,274692171162531764,2021-07,2021-07-16,2021-08-30,62250.00,5.99,252,2005-08-03,19,NaN,Serviços,GMAIL,PEQUENO,13,420995.0,417192.0,104.0
4,465309249432033993,2021-07,2021-07-05,2021-07-30,26593.95,6.99,125,2014-07-23,69,NaN,Comércio,GMAIL,GRANDE,76,324093.0,873938.0,119.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12270,705648002974742140,2021-11,2021-11-25,2021-12-13,47010.00,6.99,982,2020-01-26,67,NaN,Indústria,YAHOO,MEDIO,79,NaN,NaN,NaN
12271,4993499380140734678,2021-11,2021-11-25,2021-12-13,122875.35,8.99,565,2015-11-24,31,NaN,Serviços,GMAIL,PEQUENO,37,289980.0,311422.0,150.0
12272,4614484019183480654,2021-11,2021-11-26,2028-09-30,302200.00,5.99,138,2011-02-14,51,NaN,Indústria,YAHOO,MEDIO,93,NaN,NaN,NaN
12273,1299146298565441811,2021-11,2021-11-26,2021-12-13,143791.85,5.99,14,2000-08-15,21,NaN,Serviços,HOTMAIL,MEDIO,21,348725.0,220341.0,122.0


In [256]:
test = test.drop(columns={
    'Unnamed: 0_x',
    'Unnamed: 0_y'
})

In [257]:
test = test.loc[test['CEP_2_DIG'] != 'na']

In [258]:
# Replace de X por PF e fill de missing com PJ em FLAG_PF
test['FLAG_PF'] = test['FLAG_PF'].fillna('PJ')
test['FLAG_PF'] = test['FLAG_PF'].str.replace('X','PF')

# Preenchendo missing values de variáveis categóricas com 'SEM_VALOR'
replace_strings = 'SEM_VALOR'
test['PORTE'] = test['PORTE'].fillna(replace_strings)
test['DOMINIO_EMAIL'] = test['DOMINIO_EMAIL'].fillna(replace_strings)
test['SEGMENTO_INDUSTRIAL'] = test['SEGMENTO_INDUSTRIAL'].fillna(replace_strings)

# Preenchendo missing values de DATA_CADASTRO com mode
mode_cadastro = test['DATA_CADASTRO'].mode()[0]
test['DATA_CADASTRO'] = test['DATA_CADASTRO'].fillna(mode_cadastro)


<ipython-input-258-917ed2dd10db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['FLAG_PF'] = test['FLAG_PF'].fillna('PJ')
<ipython-input-258-917ed2dd10db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['FLAG_PF'] = test['FLAG_PF'].str.replace('X','PF')
<ipython-input-258-917ed2dd10db>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [259]:
test['SAFRA_REF'] = pd.to_datetime(test['SAFRA_REF'])
test['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(test['DATA_EMISSAO_DOCUMENTO'])
test['DATA_VENCIMENTO'] = pd.to_datetime(test['DATA_VENCIMENTO'])
test['DATA_CADASTRO'] = pd.to_datetime(test['DATA_CADASTRO'])

<ipython-input-259-af13aaddc06c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['SAFRA_REF'] = pd.to_datetime(test['SAFRA_REF'])
<ipython-input-259-af13aaddc06c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(test['DATA_EMISSAO_DOCUMENTO'])
<ipython-input-259-af13aaddc06c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [260]:
test['PRAZO'] = test['DATA_VENCIMENTO'] - test['DATA_EMISSAO_DOCUMENTO']

<ipython-input-260-53c505f35cf6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['PRAZO'] = test['DATA_VENCIMENTO'] - test['DATA_EMISSAO_DOCUMENTO']


In [261]:
'''Como os números do CEP possuem significado, vamos pegar o primeiro que
representa a Zona Postal. O segundo possui muitas variáveis dentro de cada
Zona Postal, portanto vou usar apenas o primeiro'''

primeiro_numero = []

for i in test['CEP_2_DIG']:
  primeiro = i[0]
  if primeiro == '0':
    primeiro_numero.append('Região Metropolitana de São Paulo')
  elif primeiro == '1':
      primeiro_numero.append('Litoral e interior de SP')
  elif primeiro == '2':
      primeiro_numero.append('RJ / ES')
  elif primeiro == '3':
      primeiro_numero.append('MG')
  elif primeiro == '4':
      primeiro_numero.append('BH / SE')
  elif primeiro == '5':
      primeiro_numero.append('PE / AL / PB / RN')
  elif primeiro == '6':
      primeiro_numero.append('CE / PI / MA / PA / AP / AM / RR / AC')
  elif primeiro == '7':
      primeiro_numero.append('DF / GO / RO / TO / MT / MS')
  elif primeiro == '8':
      primeiro_numero.append('PR / SC')
  elif primeiro == '9':
      primeiro_numero.append('RS')

test['ZONA_POSTAL'] = primeiro_numero

test['CEP_2_DIG'] = test['CEP_2_DIG'].astype('int')

<ipython-input-261-c4654d165eb0>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ZONA_POSTAL'] = primeiro_numero
<ipython-input-261-c4654d165eb0>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['CEP_2_DIG'] = test['CEP_2_DIG'].astype('int')


In [262]:
test['LIFETIME_CLIENTE_DIAS'] = test['DATA_EMISSAO_DOCUMENTO'] - test['DATA_CADASTRO']
test['LIFETIME_CLIENTE_DIAS'] = test['LIFETIME_CLIENTE_DIAS'].dt.days

<ipython-input-262-c6f09f2bff5b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['LIFETIME_CLIENTE_DIAS'] = test['DATA_EMISSAO_DOCUMENTO'] - test['DATA_CADASTRO']
<ipython-input-262-c6f09f2bff5b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['LIFETIME_CLIENTE_DIAS'] = test['LIFETIME_CLIENTE_DIAS'].dt.days


In [263]:
# Criando features com mês, ano e dia da semana

test['MES_EMISSAO_DOCUMENTO'] = test['DATA_EMISSAO_DOCUMENTO'].dt.month
test['MES_VENCIMENTO'] = test['DATA_VENCIMENTO'].dt.month
test['MES_CADASTRO'] = test['DATA_CADASTRO'].dt.month
test['MES_SAFRA_REF'] = test['SAFRA_REF'].dt.month

test['DIASEMANA_EMISSAO'] = test['DATA_EMISSAO_DOCUMENTO'].dt.day_name()
test['DIASEMANA_VENCIMENTO'] = test['DATA_VENCIMENTO'].dt.day_name()
test['DIASEMANA_CADASTRO'] = test['DATA_CADASTRO'].dt.day_name()
test['DIASEMANA_SAFRA_REF'] = test['SAFRA_REF'].dt.day_name()

test['ANO_EMISSAO_DOCUMENTO'] = test['DATA_EMISSAO_DOCUMENTO'].dt.year
test['ANO_VENCIMENTO'] = test['DATA_VENCIMENTO'].dt.year
test['ANO_CADASTRO'] = test['DATA_CADASTRO'].dt.year
test['ANO_SAFRA_REF'] = test['SAFRA_REF'].dt.year

<ipython-input-263-d52058579f7f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['MES_EMISSAO_DOCUMENTO'] = test['DATA_EMISSAO_DOCUMENTO'].dt.month
<ipython-input-263-d52058579f7f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['MES_VENCIMENTO'] = test['DATA_VENCIMENTO'].dt.month
<ipython-input-263-d52058579f7f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [264]:
test = test.drop(columns={'DATA_EMISSAO_DOCUMENTO','DATA_VENCIMENTO','DATA_CADASTRO'})

In [265]:
pip install category-encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [266]:
# Carregar o modelo a partir do arquivo pickle
modelo = pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/Datarisk/model/melhor_modelo.pickle', 'rb'))


In [267]:
test = test.drop(columns={'CEP_2_DIG', 'DDD', 'VALOR_A_PAGAR'})

In [268]:
test['PRAZO'] = test['PRAZO'].astype('int')

In [269]:
test.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'TAXA', 'FLAG_PF', 'SEGMENTO_INDUSTRIAL',
       'DOMINIO_EMAIL', 'PORTE', 'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS',
       'PRAZO', 'ZONA_POSTAL', 'LIFETIME_CLIENTE_DIAS',
       'MES_EMISSAO_DOCUMENTO', 'MES_VENCIMENTO', 'MES_CADASTRO',
       'MES_SAFRA_REF', 'DIASEMANA_EMISSAO', 'DIASEMANA_VENCIMENTO',
       'DIASEMANA_CADASTRO', 'DIASEMANA_SAFRA_REF', 'ANO_EMISSAO_DOCUMENTO',
       'ANO_VENCIMENTO', 'ANO_CADASTRO', 'ANO_SAFRA_REF'],
      dtype='object')

In [270]:
test['RENDA_MES_ANTERIOR'] = test['RENDA_MES_ANTERIOR'].fillna(0)
test['NO_FUNCIONARIOS'] = test['NO_FUNCIONARIOS'].fillna(0)

In [271]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12236 entries, 0 to 12274
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID_CLIENTE             12236 non-null  int64         
 1   SAFRA_REF              12236 non-null  datetime64[ns]
 2   TAXA                   12236 non-null  float64       
 3   FLAG_PF                12236 non-null  object        
 4   SEGMENTO_INDUSTRIAL    12236 non-null  object        
 5   DOMINIO_EMAIL          12236 non-null  object        
 6   PORTE                  12236 non-null  object        
 7   RENDA_MES_ANTERIOR     12236 non-null  float64       
 8   NO_FUNCIONARIOS        12236 non-null  float64       
 9   PRAZO                  12236 non-null  int64         
 10  ZONA_POSTAL            12236 non-null  object        
 11  LIFETIME_CLIENTE_DIAS  12236 non-null  int64         
 12  MES_EMISSAO_DOCUMENTO  12236 non-null  int64         
 13  M

In [272]:
prever = modelo.predict(test)

In [273]:
probs = modelo.predict_proba(test)

print(probs)

[[0.81414963 0.18585037]
 [0.88914526 0.11085474]
 [0.8550115  0.1449885 ]
 ...
 [0.67795274 0.32204726]
 [0.66934566 0.33065434]
 [0.66990291 0.33009709]]


In [274]:
test['INADIMPLENTE'] = probs[:,1]
test = test[['ID_CLIENTE','SAFRA_REF','INADIMPLENTE']]

In [275]:
test

,ID_CLIENTE,SAFRA_REF,INADIMPLENTE
0,5058298901476893676,2021-07-01,0.185850
1,274692171162531764,2021-07-01,0.110855
2,274692171162531764,2021-07-01,0.144988
3,274692171162531764,2021-07-01,0.113355
4,465309249432033993,2021-07-01,0.322010
...,...,...,...
12270,705648002974742140,2021-11-01,0.452671
12271,4993499380140734678,2021-11-01,0.119389
12272,4614484019183480654,2021-11-01,0.322047
12273,1299146298565441811,2021-11-01,0.330654


In [276]:
test.to_csv('previsoes.csv', index=False)